# Census tract map of number of daily bus arrivals by CalEnviroScreen level

* 3 maps, one for each CalEnviroScreen level of low/medium/high
* Each map plots number of bus arrivals per 1k (colored as low/medium/high)
* One specific map for CalEnviroScreen high-need to show where there is zero service

In [ ]:
import branca
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import prep_data
import map_utils

from IPython.display import Markdown

pd.set_option("display.max_rows", 20)

In [ ]:
df = prep_data.generate_stop_times_tract_data()

# Address small "numbers issue
df = df[df.Population > 10].reset_index(drop=True)

In [ ]:
sample_tracts = ["06001400100", "06001400600", "06001400700", "06001404501"]
#to_plot = df[df.Tract.isin(sample_tracts)]

In [ ]:
# If equity_group==1, corresponds to lower CalEnviroScreen scores
# If equity_group==3, corresponds to higher CalEnviroScreen scores (worse off) -- focus on these

# Focus on high density areas, then map out low/med/high service
# For calenviroscreen worst ones, low/med/high service

In [ ]:
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "County": "County",
    "City": "City",
    "equity_group": "CalEnviroScreen Group (3 is worst)", 
    "avg_density_score": "Job/Population Density Group (3 is densest)",
    "num_arrivals": "# Daily Bus Arrivals",
    "stop_id": "# Bus Stops",
    "itp_id": "# Operators",
    "arrivals_sq_mi": "Arrivals per Sq Mi",
    "arrivals_per_1k": "Arrivals per 1k",
    "arrivals_group": "Arrivals Group (3 is highest)",
}

def grab_legend_thresholds(plot_col):
    cut1 = df[(df.arrivals_group==1) & (df[plot_col] > 0)][plot_col].min()
    cut2 = df[df.arrivals_group==2][plot_col].min()
    cut3 = df[df.arrivals_group==3][plot_col].min()

    MIN_VALUE = df[plot_col].min()
    MAX_VALUE = df[plot_col].max()
    
    colormap_cutoff = [cut1, cut2, cut3]
    
    return colormap_cutoff, MIN_VALUE, MAX_VALUE
    

cmap = plt.cm.get_cmap('viridis', 4)  

COLOR_LIST = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    hex_value = matplotlib.colors.rgb2hex(rgba)
    COLOR_LIST.append(hex_value)

COLOR_LIST.reverse()

# Maybe set the colors more explicitly
# Colors
gray = "#8D9191" 
light_green = "#6EB98F"
light_blue = "#1282A2"
navy = "#0A4C6A"

#colorscale = branca.colormap.linear.Blues_09

In [ ]:
LOOP_GROUPS = {
    1: "Low", 
    2: "Medium",
    3: "High",
}

plot_col = "arrivals_per_1k"
# For indexing cut-off, we need 1 less than the number of groups
# It goes from [vmin, cut1], [cut1, cut2], [cut2, cut3]
# Not sure where the open parantheses are [) or (]?
colormap_cutoff, MIN_VALUE, MAX_VALUE = grab_legend_thresholds(plot_col)
print(colormap_cutoff)

colorscale = branca.colormap.LinearColormap(
                #colors=COLOR_LIST,
                colors=[gray, light_green, light_blue, navy], 
                index=colormap_cutoff,
                vmin=MIN_VALUE, vmax=MAX_VALUE,
)

for i, group_name in LOOP_GROUPS.items():
    to_plot = df[df.equity_group==i]
    # May need to adjust colorscale, how it shows up on map...
    display(colorscale)
    fig = map_utils.make_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = popup_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 500, fig_height = 600, 
                                    zoom=6, centroid = [35.8, -119.4],
                                    title=f"Bus Service Levels for {group_name} CalEnviroScreen Tracts",
                                    #zoom=12, centroid = [37.84, -122.27]
                                   )
    fig.save(f"./img/arrivals_pc_{group_name.lower()}.html")
    

In [ ]:
plot_col = "num_arrivals"
# For indexing cut-off, we need 1 less than the number of groups
# It goes from [vmin, cut1], [cut1, cut2], [cut2, cut3]
# Not sure where the open parantheses are [) or (]?
colormap_cutoff, MIN_VALUE, MAX_VALUE = grab_legend_thresholds(plot_col)
print(colormap_cutoff)

colorscale = branca.colormap.LinearColormap(
                #colors=COLOR_LIST,
                colors=[gray, light_green, light_blue, navy], 
                index=colormap_cutoff,
                vmin=MIN_VALUE, vmax=MAX_VALUE,
)

for i, group_name in LOOP_GROUPS.items():
    to_plot = df[df.equity_group==i]
    display(colorscale)
    fig = map_utils.make_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = popup_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 500, fig_height = 600, 
                                    zoom=6, centroid = [35.8, -119.4],
                                    title=f"Bus Service Levels for {group_name} CalEnviroScreen Tracts",
                                    #zoom=12, centroid = [37.84, -122.27]
                                   )
    fig.save(f"./img/arrivals_{group_name.lower()}.html")
    

In [ ]:
to_plot = df[(df.equity_group==3) & (df.stop_id==0)]
la = to_plot[(to_plot.County=="Los Angeles")]

In [ ]:
la.head(2)

In [ ]:
# Map for where there is zero service
#https://notebook.community/racu10/emapy/notebooks/Colormaps
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "County": "County",
    "City": "City",
    "equity_group": "CalEnviroScreen Group (3 is worst)", 
    "avg_density_score": "Job/Population Density Group (3 is densest)",
}

to_plot = la
plot_col = "avg_density_score"

colorscale = branca.colormap.StepColormap(
                colors=["orange", "green", "navy"], 
                index=[1., 2., 3.],
                vmin=1, vmax=4,
)
display(colorscale)

fig = map_utils.make_choropleth_map(to_plot, 
                                plot_col = plot_col, 
                                popup_dict = popup_dict, 
                                tooltip_dict = popup_dict, 
                                colorscale = colorscale, 
                                fig_width = 500, fig_height = 600, 
                                #zoom=6, centroid = [35.8, -119.4],
                                zoom=11, centroid = [34.00, -118.18],    
                                title=f"Zero Bus Service for High-Need CalEnviroScreen Tracts",
                                #zoom=12, centroid = [37.84, -122.27]
                               )
#fig.save(f"./img/arrivals_{group_name.lower()}.html")

In [ ]:
fig